<a href="https://colab.research.google.com/github/sumitshekhar41/AIAgents/blob/main/Program_7_Utility_Based_AI_Agent_(with_Optional_OpenAI_for_Natural_Advice).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####**Explanation:**

* Teaches students how to **assign value** to actions
* Shows **rational choice making**



In [1]:
# === Mount Google Drive to access API key ===
from google.colab import drive
drive_path = '/content/drive'
drive.mount(drive_path)

Mounted at /content/drive


In [2]:
import json

secret_file_path = "/content/drive/My Drive/Secret_Keys/OpenAI_Secret_Key.json"

with open(secret_file_path, "r") as f:
    data = json.load(f)

secret_Key = data["openai_api_key"]

In [ ]:
# Step 1: Get game state input from user
def get_game_state():
    print("Enter current game state values (0–100):\n")

    while True:
        try:
            health = float(input("Player Health (0–100): "))
            enemy_distance = float(input("Enemy Distance (0–100): "))
            resources = float(input("Available Resources (0–100): "))

            if all(0 <= val <= 100 for val in [health, enemy_distance, resources]):
                return health, enemy_distance, resources
            else:
                print("All values must be between 0 and 100.")
        except ValueError:
            print("Please enter valid numeric inputs.")

# Step 2: Compute utility for each possible action
def compute_utilities(health, enemy_distance, resources):
    utilities = {}

    # Attack: High utility if health is good, enemy is close, and resources are available
    utilities["attack"] = (
        0.4 * (100 - enemy_distance) +
        0.3 * health +
        0.3 * resources
    )

    # Defend: Higher utility when health is moderate and enemy is close
    utilities["defend"] = (
        0.4 * (100 - health) +
        0.4 * (100 - enemy_distance) +
        0.2 * resources
    )

    # Retreat: Higher utility when health is low and enemy is close
    utilities["retreat"] = (
        0.5 * (100 - health) +
        0.4 * (100 - enemy_distance) +
        0.1 * (100 - resources)
    )

    return utilities

# Step 3: Pick the best action
def select_best_action(utilities):
    print("\nComputed Utility Scores:")
    for action, score in utilities.items():
        print(f"{action.ljust(10)} -> {score:.2f}")

    best = max(utilities, key=utilities.get)
    print(f"\nSelected Action: {best}")
    return best

# Step 4: Ask GPT-4o-mini to explain the selected action
def explain_action_with_openai(action, health, enemy_distance, resources):
    from openai import OpenAI
    client = OpenAI(api_key=secret_Key)  # Replace with your key

    prompt = (
        f"As a game AI, I chose to '{action}' based on the following:\n"
        f"- Player health: {health}\n"
        f"- Enemy distance: {enemy_distance}\n"
        f"- Available resources: {resources}\n"
        f"\nExplain why this was the best decision."
    )

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a game AI coach that explains decisions clearly."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

# Step 5: Run everything
def run_game_ai_agent():
    print("=== Game AI: Utility-Based Combat Agent ===\n")
    health, enemy_distance, resources = get_game_state()
    utilities = compute_utilities(health, enemy_distance, resources)
    best_action = select_best_action(utilities)

    use_openai = True
    if use_openai:
        print("\nExplanation from GPT-4o-mini:\n")
        explanation = explain_action_with_openai(best_action, health, enemy_distance, resources)
        print(explanation)

# Start the agent
run_game_ai_agent()